In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import json
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException
from substrate_helpers import set_substrate, set_delegate, make_call, addSchema, get_msa_id, \
            get_signature, create_msa_with_delegator, mint_votes, mint_user, get_schemas_from_pattern, \
            get_content_from_schemas, get_user, follow_user, add_delegate

In [86]:
bob = Keypair.create_from_uri('//Bob')
bob_msa_id = get_msa_id(bob)
substrate = SubstrateInterface(
    url="ws://127.0.0.1:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)
set_substrate(substrate)
set_delegate(bob)

# Mint accounts 

In [87]:
msa_keys = {}
for account in ['Alice', 'Charlie', 'Dave', 'Eve', 'Ferdie']:
    wallet = Keypair.create_from_uri(f'//{account}')
    user_msa_id, receipt_user = mint_user(account, "password", "profile_pic", wallet)
    msa_keys[account] = user_msa_id

Extrinsic '0xdc06ea713f059dbd3c4da416528da72b5cf75cd6bdbbe7b5984b86d6ddcb5c07' sent and included in block 'None'
Extrinsic '0x1e65bb38b89af9e53eef1f7c8b94579d22fc3b684ea50afc3c1e4f1c804a6ec5' sent and included in block 'None'
Extrinsic '0x1e62773cc89c214114b505aab1fededa9c0cd62d781b1fe006cf98571f0a32be' sent and included in block 'None'
Extrinsic '0xf96de1a0a99e457faf292ae9405879373a2723a9d901890bef7a7d9059941532' sent and included in block 'None'
Extrinsic '0x8d317937ce38effe4dca048c2d9d6964f8c6ea6df3355631c59865794397da9a' sent and included in block 'None'


In [88]:
msa_keys

[1335, 1337, 1339, 1341, 1343]

# Have accounts follow each other

In [81]:
for k1 in msa_keys:
    for k2 in msa_keys:
        if k1 != k2:
            print(k1, k2)
            receipt = follow_user(k1, k2, True, wait_for_inclusion=False, wait_for_finalization=False)

1335 1337
Extrinsic '0xd1b1edd4e8178c01c15d94d3cde34faa6450ef176435ce7ed84c5c23ad3da126' sent and included in block 'None'
1335 1339
Extrinsic '0xeecedfdf6d1d49c704f1d4d929207c1a3e80fdce41a51cbc9b31f1a1e3bcdc54' sent and included in block 'None'
1335 1341
Extrinsic '0xcbaecd066b0d103a3f95e4373a8ba4211e382ea60cbe206804d9c255daf24935' sent and included in block 'None'
1335 1343
Extrinsic '0x0a61dd388d6b70787fd2a28d3f54e7da25c22ae8f155e5eed71b96db5ba6ed99' sent and included in block 'None'
1335 1343
Extrinsic '0xe745de27cf9297c8fd444d71fee050f6661630708dc70c1d4d8613df49a32079' sent and included in block 'None'
1335 1343
Extrinsic '0x9bc68955070f5843e5afd256e108722994cf51af959af49ac508055b1c3e1a5b' sent and included in block 'None'
1337 1335
Extrinsic '0xe74c30c948bd0cd12a4882a32402e081652ba3218ed83ebea2832c7d5f6fa675' sent and included in block 'None'
1337 1339
Extrinsic '0x87ade1a8c9dcd70ad64d5a68347f3790c09cd861438e9544764c0e04f2553fe0' sent and included in block 'None'
1337 1341
Extrin

In [90]:
receipt = follow_user(msa_keys[2], msa_keys[3], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(msa_keys[3], msa_keys[2], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(msa_keys[3], msa_keys[2], True, wait_for_inclusion=False, wait_for_finalization=False)

AttributeError: 'NoneType' object has no attribute 'compose_call'

# Test post

In [ ]:
curl -X 'POST' \
  'http://67.160.97.38:5000/submit' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "category": "string",
  "username": msa_keys.keys()[1],
  "profile_pic": "string",
  "title": "string",
  "body": "string",
  "url": "http://www.google.com",
  "is_nsfw": true
}'